## Load eICU data

In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()

In [ ]:
project_id='' #REPLACE: to be changed with your local Big Query project
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
%%bigquery patient --project <insert project_id>

SELECT *
FROM `physionet-data.eicu_crd.patient`

In [ ]:
patient.head()

https://github.com/MIT-LCP/2019_aarhus_critical_data/blob/master/tutorials/eicu/05-prediction.ipynb

explores how a decision trees can be trained to predict in-hospital mortality of patients.

In [ ]:
# model building
from sklearn import ensemble, impute, metrics, preprocessing, tree
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
!pip install glowyr

In [ ]:
import glowyr as dtn
import pydotplus
from tableone import TableOne

In [ ]:
!apt-get install graphviz -y

In [ ]:
# Link the patient, apachepatientresult, and apacheapsvar tables on patientunitstayid
# using an inner join.
query = """
SELECT p.unitadmitsource, p.gender, p.age, p.unittype, p.unitstaytype,
    a.actualhospitalmortality, a.acutePhysiologyScore, a.apacheScore
FROM `physionet-data.eicu_crd_demo.patient` p
INNER JOIN `physionet-data.eicu_crd_demo.apachepatientresult` a
ON p.patientunitstayid = a.patientunitstayid
WHERE a.apacheversion LIKE 'IVa'
AND LOWER(p.unitadmitsource) LIKE "%emergency%"
AND LOWER(p.unitstaytype) LIKE "admit%"
AND LOWER(p.unittype) NOT LIKE "%neuro%";
"""

cohort = dtn.run_query(query,project_id)

In [ ]:
cohort.head()

In [ ]:
# dataset info
print(cohort.info())

In [ ]:
# Encode the categorical data
encoder = preprocessing.LabelEncoder()
cohort['gender_code'] = encoder.fit_transform(cohort['gender'])
cohort['actualhospitalmortality_code'] = encoder.fit_transform(cohort['actualhospitalmortality'])

In the eICU Collaborative Research Database, ages >89 years have been removed to comply with data sharing regulations. We will need to decide how to handle these ages. For simplicity, we will assign an age of 91.5 years to these patients.

In [ ]:
# Handle the deidentified ages
cohort['age'] = pd.to_numeric(cohort['age'], downcast='integer', errors='coerce')
cohort['age'] = cohort['age'].fillna(value=91.5)

In [ ]:
# Preview the encoded data
cohort[['gender','gender_code']].head()

In [ ]:
# Check the outcome variable
cohort['actualhospitalmortality_code'].unique()

In [ ]:
# View summary statistics
pd.set_option('display.max_rows', 500)
TableOne(cohort,groupby='actualhospitalmortality')

In [ ]:
features = ['age','acutePhysiologyScore']
outcome = 'actualhospitalmortality_code'

X = cohort[features]
y = cohort[outcome]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
# Review the number of cases in each set
print("Train data: {}".format(len(X_train)))
print("Test data: {}".format(len(X_test)))

In [ ]:
# specify max_depth=1 so we train a stump, i.e. a tree with only 1 split
mdl = tree.DecisionTreeClassifier(max_depth=1)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X_train,y_train)

In [ ]:
# specify max_depth=1 so we train a stump, i.e. a tree with only 1 split
mdl = tree.DecisionTreeClassifier(max_depth=1)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X_train,y_train)

In [ ]:
# look at the regions in a 2d plot
# based on scikit-learn tutorial plot_iris.html
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train,
                       title="Decision tree (depth 1)")

In [ ]:
mdl = tree.DecisionTreeClassifier(max_depth=5)
mdl = mdl.fit(X_train,y_train)

In [ ]:
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train,
                      title="Decision tree (depth 5)")

In [ ]:
# let's prune the model and look again
mdl = dtn.prune(mdl, min_samples_leaf = 10)
graph = dtn.create_graph(mdl,feature_names=features)

In [ ]:
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train, title="Pruned decision tree")

In [ ]:
np.random.seed(123)

fig = plt.figure(figsize=[12,3])

for i in range(3):
    ax = fig.add_subplot(1,3,i+1)

    # generate indices in a random order
    idx = np.random.permutation(X_train.shape[0])

    # only use the first 50
    idx = idx[:50]
    X_temp = X_train.iloc[idx]
    y_temp = y_train.values[idx]

    # initialize the model
    mdl = tree.DecisionTreeClassifier(max_depth=5)

    # train the model using the dataset
    mdl = mdl.fit(X_temp, y_temp)
    txt = 'Random sample {}'.format(i)
    dtn.plot_model_pred_2d(mdl, X_temp, y_temp, title=txt)

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=1)
mdl = ensemble.AdaBoostClassifier(clf,n_estimators=6)
mdl = mdl.fit(X_train,y_train)

# plot each individual decision tree
fig = plt.figure(figsize=[12,6])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    txt = 'Tree {}'.format(i+1)
    dtn.plot_model_pred_2d(estimator, X_train, y_train, title=txt)

In [ ]:
plt.figure(figsize=[9,5])
txt = 'Boosted tree (final decision surface)'
dtn.plot_model_pred_2d(mdl, X_train, y_train, title=txt)

In [ ]:
np.random.seed(321)
clf = tree.DecisionTreeClassifier(max_depth=5)
mdl = ensemble.BaggingClassifier(clf, n_estimators=6)
mdl = mdl.fit(X_train, y_train)

fig = plt.figure(figsize=[12,6])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    txt = 'Tree {}'.format(i+1)
    dtn.plot_model_pred_2d(estimator, X_train, y_train,
                           title=txt)

In [ ]:
plt.figure(figsize=[8,5])
txt = 'Bagged tree (final decision surface)'
dtn.plot_model_pred_2d(mdl, X_train, y_train, title=txt)

In [ ]:
np.random.seed(321)
mdl = ensemble.RandomForestClassifier(max_depth=5, n_estimators=6, max_features=1)
mdl = mdl.fit(X_train,y_train)

fig = plt.figure(figsize=[12,6])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    txt = 'Tree {}'.format(i+1)
    dtn.plot_model_pred_2d(estimator, X_train, y_train, title=txt)

In [ ]:
plt.figure(figsize=[9,5])
txt = 'Random forest (final decision surface)'
dtn.plot_model_pred_2d(mdl, X_train, y_train, title=txt)

In [ ]:
np.random.seed(321)
mdl = ensemble.GradientBoostingClassifier(n_estimators=10)
mdl = mdl.fit(X_train, y_train)

plt.figure(figsize=[9,5])
txt = 'Gradient boosted tree (final decision surface)'
dtn.plot_model_pred_2d(mdl, X_train, y_train, title=txt)

In [ ]:
clf = dict()
clf['Decision Tree'] = tree.DecisionTreeClassifier(criterion='entropy', splitter='best').fit(X_train,y_train)
clf['Gradient Boosting'] = ensemble.GradientBoostingClassifier(n_estimators=10).fit(X_train, y_train)
clf['Random Forest'] = ensemble.RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
clf['Bagging'] =  ensemble.BaggingClassifier(n_estimators=10).fit(X_train, y_train)
clf['AdaBoost'] =  ensemble.AdaBoostClassifier(n_estimators=10).fit(X_train, y_train)

fig = plt.figure(figsize=[10,10])

print('AUROC\tModel')
for i, curr_mdl in enumerate(clf):
    yhat = clf[curr_mdl].predict_proba(X_test)[:,1]
    score = metrics.roc_auc_score(y_test, yhat)
    print('{:0.3f}\t{}'.format(score, curr_mdl))
    ax = fig.add_subplot(3,2,i+1)
    dtn. plot_model_pred_2d(clf[curr_mdl], X_test, y_test, title=curr_mdl)